In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = os.getenv("LANGCHAIN_ENDPOINT")
LANGCHAIN_PROJECT_ID = os.getenv("LANGCHAIN_PROJECT_ID")

In [4]:
!pip install langchain_community langchain_openai langchain_chroma

  Using cached protobuf-5.29.3-cp310-abi3-win_amd64.whl.metadata (592 bytes)
Using cached protobuf-5.29.3-cp310-abi3-win_amd64.whl (434 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5


In [8]:
from langchain_community.document_loaders import TextLoader

loaders = [
    # 파일을 로드합니다.
    TextLoader(r"C:\Users\eys63\GitHub\JJU\01-이연승\data\241227_appendix-keywords.txt"),
]

docs = []
for loader in loaders:
    # 로더를 사용하여 문서를 로드하고 docs 리스트에 추가합니다.
    docs.extend(loader.load())

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 자식 분할기를 생성합니다.
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

In [12]:
from langchain_openai import OpenAIEmbeddings
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma

# DB를 생성합니다.
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=OpenAIEmbeddings()
)

store = InMemoryStore()

In [13]:
from langchain.retrievers import ParentDocumentRetriever

# Retriever 를 생성합니다.
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

In [14]:
# 문서를 검색기에 추가합니다. docs는 문서 목록이고, ids는 문서의 고유 식별자 목록입니다.
retriever.add_documents(docs, ids=None, add_to_docstore=True)

In [15]:
# 저장소의 모든 키를 리스트로 반환합니다.
list(store.yield_keys())

['831d24db-30d6-43f7-9ea1-186c27a9467d']

In [17]:
# 유사도 검색을 수행합니다.
sub_docs = vectorstore.similarity_search("Word2Vec")

In [18]:
# sub_docs 리스트의 첫 번째 요소의 page_content 속성을 출력합니다.
print(sub_docs[0].page_content)

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성


In [19]:
# 문서를 검색하여 가져옵니다.
retrieved_docs = retriever.invoke("Word2Vec")

In [20]:
# 검색된 문서의 문서의 페이지 내용의 길이를 출력합니다.
print(
    f"문서의 길이: {len(retrieved_docs[0].page_content)}",
    end="\n\n=====================\n\n",
)

# 문서의 일부를 출력합니다.
print(retrieved_docs[0].page_content[2000:2500])


문서의 길이: 5733


 컴퓨팅을 도입하여 데이터 저장과 처리를 혁신하는 것은 디지털 변환의 예입니다.
연관키워드: 혁신, 기술, 비즈니스 모델

Crawling

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)

정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을


In [22]:
# 부모 문서를 생성하는 데 사용되는 텍스트 분할기입니다.
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)

# 자식 문서를 생성하는 데 사용되는 텍스트 분할기입니다.
# 부모보다 작은 문서를 생성해야 합니다.
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

# 자식 청크를 인덱싱하는 데 사용할 벡터 저장소입니다.
vectorstore = Chroma(
    collection_name="split_parents", embedding_function=OpenAIEmbeddings()
)
# 부모 문서의 저장 계층입니다.
store = InMemoryStore()

In [23]:
retriever = ParentDocumentRetriever(

    # 벡터 저장소를 지정합니다.
    vectorstore=vectorstore,

    # 문서 저장소를 지정합니다.
    docstore=store,

    # 하위 문서 분할기를 지정합니다.
    child_splitter=child_splitter,
    
    # 상위 문서 분할기를 지정합니다.
    parent_splitter=parent_splitter,
)

In [24]:
# 문서를 retriever에 추가합니다.
retriever.add_documents(docs)  